In [1]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import glob
import os

In [28]:
print('開始する都道府県コードを入力してください')
start_pref = input()
print(f'開始する都道府県コード : {start_pref}')

print('終了する都道府県コードを入力してください')
finish_pref = input()
print(f'終了する都道府県コー : {finish_pref}')



#座標系の変換
def crs_trf(gdf, dst_proj):
  if gdf.crs is None:
      gdf.set_crs(epsg=dst_proj, inplace=True)
  else:
      gdf.to_crs(epsg=dst_proj, inplace=True)


%cd "G:/マイドライブ/akiyamalab/卒論/負担率総集編/data"
pref = int(start_pref)
while pref <= int(finish_pref):
    
    #住宅地図と浸水ポリゴンの読み込み
    bldg_path = f'./住宅地図兼マイクロ人口/{pref}_step10.csv'
    bldg = pd.read_csv(bldg_path)
    
    #浸水ポリゴンのshpファイルの階層が都道府県ごとに違うため各ディレクトリを探索
    directory_name = f'./flood_polygon/H24_flood_polygon/{pref}'  # ディレクトリ名
    # globを使用してディレクトリ内の全ての.shpファイルを検索
    # **は全てのディレクトリを再帰的に検索するために使用され、*.shpは.shpファイルにマッチします
    flood_polygon_path = glob.glob(os.path.join(directory_name, '**', '*.shp'), recursive=True)

    flood_polygon = gpd.read_file(flood_polygon_path)
    
    
    #浸水ポリゴンのカラムを限定する
    flood_polygon = flood_polygon[['A31_001', 'geometry']]
    
    
    
    #住宅地図にgeometryを追加
    
    # ベクトル化された操作を使用してPointオブジェクトのリストを作成（applyを使うよりも効率が良い）
    points = [Point(lon, lat) for lon, lat in zip(bldg['x'], bldg['y'])]
    # PointのリストからGeoSeriesを作成
    bldg['geometry'] = gpd.GeoSeries(points)
    bldg = gpd.GeoDataFrame(bldg, geometry='geometry')
    
    
    
    
    #座標系の変換
    crs_trf(bldg, 4326)
    crs_trf(flood_polygon, 4326)
    
    
    
    #住宅地図と浸水ポリゴンの結合
    bldg_flood_rank = gpd.sjoin(bldg, flood_polygon, op='within', how='left')
    #浸水ランクと被る建物を抽出
    bldg_flood_rank = bldg_flood_rank[~bldg_flood_rank['A31_001'].isna()]
    
    
    
    
    #geometryの削除
    #bldg_flood_rank.drop(columns=['geometry'], inplace=True)
    
    
    
    
    #住宅地図の出力
    bldg_flood_rank_path = f'./mpc/{pref}.csv'
    bldg_flood_rank.to_csv(bldg_flood_rank_path, index=False)
    
    pref += 1

開始する都道府県コードを入力してください
開始する都道府県コード : 30
終了する都道府県コードを入力してください
終了する都道府県コー : 30
G:\マイドライブ\akiyamalab\卒論\負担率総集編\data


c:\Users\imai_14\anaconda3\envs\microbase\lib\site-packages\IPython\core\interactiveshell.py:3338: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


In [30]:
import os
import glob

# .shpファイルのパスを保存するためのリスト
shp_files = []

# 1から47までのディレクトリをループ
for i in range(1, 48):
    directory_name = f'G:/マイドライブ/akiyamalab/卒論/負担率総集編/data/flood_polygon/H24_flood_polygon/{str(i)}'  # ディレクトリ名
    # globを使用してディレクトリ内の全ての.shpファイルを検索
    # **は全てのディレクトリを再帰的に検索するために使用され、*.shpは.shpファイルにマッチします
    files = glob.glob(os.path.join(directory_name, '**', '*.shp'), recursive=True)
    shp_files.extend(files)  # 見つかったファイルのパスをリストに追加

# 結果を出力
for file in shp_files:
    print(file)

G:/マイドライブ/akiyamalab/卒論/負担率総集編/data/flood_polygon/H24_flood_polygon/1\A31-12_01.shp
G:/マイドライブ/akiyamalab/卒論/負担率総集編/data/flood_polygon/H24_flood_polygon/2\A31-12_02.shp
G:/マイドライブ/akiyamalab/卒論/負担率総集編/data/flood_polygon/H24_flood_polygon/3\A31-12_03.shp
G:/マイドライブ/akiyamalab/卒論/負担率総集編/data/flood_polygon/H24_flood_polygon/4\A31-12_04.shp
G:/マイドライブ/akiyamalab/卒論/負担率総集編/data/flood_polygon/H24_flood_polygon/5\A31-12_05.shp
G:/マイドライブ/akiyamalab/卒論/負担率総集編/data/flood_polygon/H24_flood_polygon/6\A31-12_06_GML\A31-12_06.shp
G:/マイドライブ/akiyamalab/卒論/負担率総集編/data/flood_polygon/H24_flood_polygon/7\A31-12_07_GML\A31-12_07.shp
G:/マイドライブ/akiyamalab/卒論/負担率総集編/data/flood_polygon/H24_flood_polygon/8\A31-12_08.shp
G:/マイドライブ/akiyamalab/卒論/負担率総集編/data/flood_polygon/H24_flood_polygon/9\A31-12_09.shp
G:/マイドライブ/akiyamalab/卒論/負担率総集編/data/flood_polygon/H24_flood_polygon/10\A31-12_10.shp
G:/マイドライブ/akiyamalab/卒論/負担率総集編/data/flood_polygon/H24_flood_polygon/11\A31-12_11.shp
G:/マイドライブ/akiyamalab/卒論/負担率総集編/data/flood_poly

In [ ]:
15,16,20,21,24,30,32,34,40,41,43,44,46

In [11]:
bldg.iloc[0:1].to_csv('G:/マイドライブ/akiyamalab/卒論/負担率総集編/data/住宅地図sample.csv',index=False)

In [12]:
bldg.iloc[0:1]

,tatemon_id,x,y,floor,AREA,atype2,total_area,KEY_CODE,estimated_structure,estimated_age,...,f_age105,num_child,num_adult,meshcode,T30_P03_SI,T30_P06_SI,T50_P02_SI,T50_P05_SI,T50_P10_SI,T50_P39_SI
0,38,133.747186,34.755088,0,139.7,1364,139.7,331012560,1,3,...,0.0,0,2.0,5233150942,5.2,5.0,5.3,5.1,5.0,4.6


In [37]:
bldg_flood_rank[bldg_flood_rank['A31_001'].isna()]

,tatemon_id,x,y,floor,AREA,atype2,total_area,KEY_CODE,estimated_structure,estimated_age,...,num_adult,meshcode,T30_P03_SI,T30_P06_SI,T50_P02_SI,T50_P05_SI,T50_P10_SI,T50_P39_SI,index_right,A31_001
0,38,133.747186,34.755088,0,139.700,1364,139.700,331012560,1,3,...,2.0,5233150942,5.2,5.0,5.3,5.1,5.0,4.6,NaN,NaN
1,45,133.748102,34.756304,0,193.125,1364,193.125,331012470,1,3,...,2.0,5233150944,5.2,5.0,5.3,5.1,5.0,4.6,NaN,NaN
2,61,133.747555,34.771666,0,270.260,1364,270.260,331012470,1,1,...,1.0,5233152942,5.1,5.0,5.2,5.1,5.0,4.6,NaN,NaN
3,62,133.747617,34.772233,0,166.450,1364,166.450,331012470,1,4,...,1.0,5233152942,5.1,5.0,5.2,5.1,5.0,4.6,NaN,NaN
4,73,133.749654,34.756339,0,128.655,1364,128.655,331012560,1,2,...,3.0,5233150944,5.2,5.0,5.3,5.1,5.0,4.6,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
338975,20807,133.840021,34.891195,0,343.190,1364,343.190,33681057003,1,3,...,1.0,5233266733,5.1,4.9,5.2,5.0,4.9,4.5,NaN,NaN
338976,20814,133.839066,34.892351,0,149.270,1364,149.270,33681057003,1,1,...,1.0,5233267711,5.1,4.9,5.2,5.0,4.9,4.5,NaN,NaN
338977,20841,133.845370,34.898555,0,265.020,1364,265.020,33681057003,1,3,...,1.0,5233267743,5.1,4.9,5.2,5.0,4.9,4.5,NaN,NaN
338978,20957,133.848981,34.880249,0,56.160,1364,56.160,33681056006,1,5,...,4.0,5233265742,5.1,4.9,5.2,5.0,4.9,4.5,NaN,NaN


In [25]:
test = 'G:/マイドライブ/akiyamalab/卒論/負担率総集編/data/損害額data/33_step8_1.csv'
test = pd.read_csv(test)

In [32]:
bldg

,tatemon_id,x,y,floor,AREA,atype2,total_area,KEY_CODE,estimated_structure,estimated_age,...,num_child,num_adult,meshcode,T30_P03_SI,T30_P06_SI,T50_P02_SI,T50_P05_SI,T50_P10_SI,T50_P39_SI,geometry
0,38,133.747186,34.755088,0,139.700,1364,139.700,331012560,1,3,...,0,2.0,5233150942,5.2,5.0,5.3,5.1,5.0,4.6,POINT (133.74719 34.75509)
1,45,133.748102,34.756304,0,193.125,1364,193.125,331012470,1,3,...,0,2.0,5233150944,5.2,5.0,5.3,5.1,5.0,4.6,POINT (133.74810 34.75630)
2,61,133.747555,34.771666,0,270.260,1364,270.260,331012470,1,1,...,0,1.0,5233152942,5.1,5.0,5.2,5.1,5.0,4.6,POINT (133.74756 34.77167)
3,62,133.747617,34.772233,0,166.450,1364,166.450,331012470,1,4,...,1,1.0,5233152942,5.1,5.0,5.2,5.1,5.0,4.6,POINT (133.74762 34.77223)
4,73,133.749654,34.756339,0,128.655,1364,128.655,331012560,1,2,...,0,3.0,5233150944,5.2,5.0,5.3,5.1,5.0,4.6,POINT (133.74965 34.75634)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
338975,20807,133.840021,34.891195,0,343.190,1364,343.190,33681057003,1,3,...,0,1.0,5233266733,5.1,4.9,5.2,5.0,4.9,4.5,POINT (133.84002 34.89119)
338976,20814,133.839066,34.892351,0,149.270,1364,149.270,33681057003,1,1,...,0,1.0,5233267711,5.1,4.9,5.2,5.0,4.9,4.5,POINT (133.83907 34.89235)
338977,20841,133.845370,34.898555,0,265.020,1364,265.020,33681057003,1,3,...,0,1.0,5233267743,5.1,4.9,5.2,5.0,4.9,4.5,POINT (133.84537 34.89855)
338978,20957,133.848981,34.880249,0,56.160,1364,56.160,33681056006,1,5,...,1,4.0,5233265742,5.1,4.9,5.2,5.0,4.9,4.5,POINT (133.84898 34.88025)


In [34]:
test[~test['A31_001'].isna()]

,tatemon_id,x,y,floor,AREA,atype2,total_area,KEY_CODE,estimated_structure,estimated_age,A31_001
0,1573,133.797213,34.672247,0,166.805,1364,166.805,331012270,1,4,12
1,1574,133.797034,34.671509,0,131.640,1364,131.640,331012270,1,2,12
2,1695,133.797357,34.672400,0,96.400,1364,96.400,331012270,1,3,12
3,1688,133.797788,34.673460,0,184.710,1364,184.710,331012270,1,3,11
4,1694,133.797064,34.672907,0,102.940,1364,102.940,331012270,1,1,11
...,...,...,...,...,...,...,...,...,...,...,...
333097,19877,133.817980,34.858477,0,253.700,1364,253.700,33681059005,1,6,12
333098,19890,133.816148,34.860425,0,107.045,1364,107.045,33681059005,1,3,11
333099,19898,133.817725,34.860710,0,118.770,1364,118.770,33681059006,1,1,11
333100,19919,133.818189,34.857995,0,162.060,1364,162.060,33681059005,1,4,13


In [30]:
flood_polygon

,A31_001,geometry
0,11,"POLYGON ((133.75753 35.05878, 133.75756 35.058..."
1,11,"POLYGON ((133.96779 34.76797, 133.96792 34.768..."
2,11,"POLYGON ((133.69167 35.07914, 133.69199 35.079..."
3,11,"POLYGON ((133.96771 34.76104, 133.96773 34.761..."
4,11,"POLYGON ((133.69323 35.09145, 133.69323 35.091..."
...,...,...
8576,13,"POLYGON ((133.99591 34.68168, 133.99591 34.681..."
8577,12,"POLYGON ((133.95190 34.63850, 133.95247 34.638..."
8578,13,"POLYGON ((133.99263 34.63999, 133.99263 34.640..."
8579,12,"POLYGON ((133.93794 34.64107, 133.93777 34.640..."
